In [1]:
from functools import partial, singledispatch
from sympy import sympify
from sympy import integrate as _integrate
from textwrap import dedent
from mpmath import e
import sympy

sympy.init_printing()

def _sum(left_or_right, y_values, delta_x):
    """
    Return the left or right hand sum of a list of y values.
    """
    # make sure y_values is a list
    y_values = list(y_values)
    
    if left_or_right == 'left':
        y_values = y_values[:-1]
    elif left_or_right == 'right':
        y_values = y_values[1:]
        
    return sum(y*delta_x for y in y_values)

lhs = partial(_sum, 'left')
rhs = partial(_sum, 'right')

@singledispatch
def integrate():
    pass

@integrate.register(dict)        
def _(observations, a_b=None, n=None, verbose=False):
    """
    Given a table of x, y values as a dictionary
    return the definite integral.
    """
    x_vals = list(observations.keys())
    delta_x = x_vals[1]-x_vals[0]
    
    # n is the number of times we evenly split the observations
    # by default, we'll choose the maximum amount of splits possible
    
    n = len(observations)-1 if n is None else n
    
    # let a and b be the largest and smallest values
    # in our list of x values by default
    
    if a_b is not None:
        a, b = a_b
    else:
        a = x_vals[0]
        b = x_vals[-1]
    
    # create the list of y values that we will compute
    # the right and left hand sums of
    start = x_vals.index(a)
    stop = x_vals.index(b)+1
    step = len(observations) // n    
    y_vals = list(observations.values())[start:stop:step]
    
    # return the average of the left and right hand sums
    left_hand_sum = lhs(y_vals, delta_x)
    right_hand_sum = rhs(y_vals, delta_x)
    result = (left_hand_sum + right_hand_sum) / 2
    
    if verbose:
        print(dedent(f"""
        inputs: {observations}
        
        a: {a}
        b: {b}
        n: {n}
        delta x: {delta_x}
        left hand sum: {left_hand_sum}
        right hand sum: {right_hand_sum}
        
        result: {result}
        """))
    
    return result

@integrate.register(sympy.Basic)
def _(expr, a_b=None):
    """
    Return the integral of a single-variable expression.
    """
    # ensure we replace e variable with constant e in
    # expression
    expr = expr.subs('e', e)
    
    
    # extract symbol
    if a_b is not None:
        a, b = a_b
        var = list(expr.free_symbols)[0]
        return _integrate(expr, (var, a, b))
    
    return _integrate(expr)

@integrate.register(str)
def _(expr, a_b=None):
    """
    Return the integral of a single-variable expression.
    """
    return integrate(sympify(expr), a_b=a_b)
    
    

In [2]:
observations = {20: 5, 22: 7, 24: 11, 26: 18, 28: 29, 30: 45}

integrate(observations, verbose=True) # -> 180 


inputs: {20: 5, 22: 7, 24: 11, 26: 18, 28: 29, 30: 45}

a: 20
b: 30
n: 5
delta x: 2
left hand sum: 140
right hand sum: 220

result: 180.0



In [3]:
integrate('2*x') # -> x**2

integrate('2*x', (0, 10)) # -> 100